In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torch.autograd import Variable

In [2]:
df=pd.read_csv('ratings.csv',names=['userID','movieID','rating','time'])
df.drop('time',axis=1,inplace=True)
users=[k for k,v in df['userID'].value_counts().iteritems() if v>5]
movies=[k for k,v in df['movieID'].value_counts().iteritems() if v>10]
df=df[(df['userID'].isin(users)) & (df['movieID'].isin(movies))]
df=df.pivot(index='userID',columns='movieID',values='rating')
print (df.shape)

(66182, 14162)


In [3]:
mask=df.copy()
mask[~mask.isnull()] = 1  # not nan
mask[mask.isnull()] = 0   # nan
df[df.isnull()] = 0   # nan

In [4]:
df_matrix=df.values
mask_matrix=mask.values
train_matrix=df_matrix[0:50000]
val_matrix=df_matrix[50000:60000]
test_matrix=df_matrix[60000:]
train_mask=mask_matrix[0:50000]
val_mask=mask_matrix[50000:60000]
test_mask=mask_matrix[60000:]

In [5]:
class Autorec(nn.Module):
    def __init__(self, hidden_size, input_size):
        super(Autorec, self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        
        self.encoder=nn.Linear(self.input_size, self.hidden_size)
        self.decoder=nn.Linear(self.hidden_size, self.input_size)
        self.sigmoid=nn.Sigmoid()
        self.decoder.weight.data = self.encoder.weight.data.transpose(0,1)
        self.register_buffer('input', torch.zeros(input_size))
        
    def forward(self, input_ratings):
        self.input=input_ratings
        enc_out = self.encoder(input_ratings)
        dec_out = 5*self.sigmoid(self.decoder(enc_out))
        return dec_out

In [6]:
def train_minibatch(input_ratings, mask, autorec, optimizer, criterion):
    optimizer.zero_grad()
    output_ratings=autorec(input_ratings.type(torch.cuda.FloatTensor))*mask.type(torch.cuda.FloatTensor)
    loss=criterion(output_ratings,input_ratings.type(torch.cuda.FloatTensor))
    loss.backward()
    optimizer.step()
    return torch.sqrt(loss)

In [7]:
def validation(input_ratings, mask, autorec):
    with torch.no_grad():
        input_ratings=input_ratings.type(torch.cuda.FloatTensor)
        output_ratings=autorec(input_ratings)*mask.type(torch.cuda.FloatTensor)
        #loss=torch.sqrt(criterion(output_ratings,input_ratings.type(torch.cuda.FloatTensor)))
        idx=torch.nonzero(mask)
        loss=0
        for i in idx:
            loss+=((output_ratings[i[0]][i[1]]-input_ratings[i[0]][i[1]]).item())**2
    return np.sqrt(loss/idx.size(0))

In [8]:
autorec=Autorec(hidden_size=500,input_size=train_matrix.shape[1])
optimizer=optim.Adam(autorec.parameters())
criterion=nn.MSELoss()
device=torch.device('cuda')
autorec=autorec.to(device)

In [10]:
num_batches=50
val_benchmark=10
for batch in range(0,num_batches):
    running_loss=0
    input_users_val=torch.from_numpy(val_matrix).to(device).detach()
    mask_val=torch.from_numpy(val_mask).to(device).detach()
    for i in range(0,train_matrix.shape[0],50):
        #print(i)
        input_users=Variable(torch.from_numpy(train_matrix[i:i+50])).to(device)
        input_mask=torch.from_numpy(train_mask[i:i+50]).to(device)
        loss=train_minibatch(input_users, input_mask, autorec, optimizer, criterion)
        running_loss+=loss.item()
        if (i)%10000==0:
            val_loss=validation(input_users_val, mask_val, autorec)
            print ('Batch: {} | Step: {}/{} | Training Loss: {} | Validation Loss: {}'.format(batch+1,int(i/10000)+1,5,round(running_loss,4),round(val_loss,4)))
            running_loss=0
            if(val_loss<val_benchmark):
                print ('%---Saving the model---%')
                torch.save({
                    'step':i+1,
                    'autorec_state_dict': autorec.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'batch':batch,
                    'loss':val_loss
                    },'model.pth')
                val_benchmark=val_loss

Batch: 1 | Step: 1/10 | Training Loss: 0.0447 | Validation Loss: 1.9232
%---Saving the model---%
Batch: 1 | Step: 2/10 | Training Loss: 4.0059 | Validation Loss: 1.3475
%---Saving the model---%
Batch: 1 | Step: 3/10 | Training Loss: 3.3642 | Validation Loss: 1.2639
%---Saving the model---%
Batch: 1 | Step: 4/10 | Training Loss: 3.209 | Validation Loss: 1.2176
%---Saving the model---%
Batch: 1 | Step: 5/10 | Training Loss: 3.1295 | Validation Loss: 1.1834
%---Saving the model---%
Batch: 1 | Step: 6/10 | Training Loss: 3.0452 | Validation Loss: 1.1544
%---Saving the model---%
Batch: 1 | Step: 7/10 | Training Loss: 2.8858 | Validation Loss: 1.1269
%---Saving the model---%
Batch: 1 | Step: 8/10 | Training Loss: 2.9466 | Validation Loss: 1.1206
%---Saving the model---%
Batch: 1 | Step: 9/10 | Training Loss: 2.8858 | Validation Loss: 1.1026
%---Saving the model---%
Batch: 1 | Step: 10/10 | Training Loss: 2.8 | Validation Loss: 1.0962
%---Saving the model---%
Batch: 2 | Step: 1/10 | Training 

KeyboardInterrupt: 